### Part 2:

This was figured out by luck & doing a little reminder on prime number operations. 

My approach was to start w/ the sample and try a variety of standard operations rather than `divide by 3` including:
- dividing by all integers from 1 - 1_000_000
- taking the power of `1/n` for `2 - 10`
- using log operations (really thought this would be it)
- A variety of modulus math (getting closer)

And eventually I tried a very large range of ints where I took `value % n` and noticed that a few values passed the first round (single cycle), but only 1 passed the next example (twentieth cycle). This number was `96577`, which can also be written as `23 * 19 * 13 * 17`, which is also the product of all the sample monkey's divisibility test numbers.

So, this can be extended to the actual data, which has a value of `9699690`. 

And I will be honest, I don't actually understand the mathematical significance of this, I really just stumbled on it after refreshing my memory in how primes and modulo operator work together sometimes. I need to go read up on it!

In [1]:
import math
import numpy as np
from dataclasses import dataclass, field
from matplotlib import pyplot as plt
import operator

# https://stackoverflow.com/questions/1740726/turn-string-into-operator
# Note: Wow I did not need this given how small the actual input is, oh well.
ops = {
    '+' : operator.add,
    '-' : operator.sub,
    '*' : operator.mul,
    '/' : operator.truediv,  # use operator.div for Python 2
    '%' : operator.mod,
    '^' : operator.xor,
}

#print(ops["+"](1,1)) # prints 2 


@dataclass
class monkey:
    """Class for monkey shenanigans"""
    name: int
    items: list
    operation_str: str
    test_divisor: int
    true: int
    false: int
    inspection_count: int
    anxiety_int: int
    
    def parse_operation(self):
        """First build out the rules for updating"""
        ops = self.operation_str.split(' = ')[1]
        self.x1, self.op, self.x2 = ops.split(' ')
        
    def review_items(self, monkey_list):
        """Iterate over all items and handle accordingly"""
        for item in self.items:
            self.inspection_count += 1
            self.handle_item(int(item), monkey_list)
        self.items = [] # empty list
    
    def handle_item(self, item, monkey_list):
        """
        Run logic for single item, pass off to proper monkey
        """
        if self.x1 == 'old':
            x1 = item
        else:
            x1 = int(self.x1)
        if self.x2 == 'old':
            x2 = item
        else:
            x2 = int(self.x2)
        
        # apply operation
        new = ops[self.op](x1, x2)
        
        # reduce anxiety -> new approach
        div = new % self.anxiety_int
        
        # handle test divisor: where to pass
        if div % self.test_divisor == 0:
            monkey_list[self.true].items.append(div)     
        else:
            #print(f"Not divisible: Pass off {div} to monkey {self.false}")
            monkey_list[self.false].items.append(div)

In [2]:
# read sample data
from collections import defaultdict
with open('data/day11_sample.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

input_monkey = defaultdict(list)
i = 0
for line in data:
    if line == '':
        i += 1
    else:
        input_monkey[i].append(line)

In [3]:
# The search
test_d_list = []
for divider in range(1,100_000): # limiting for brevity
    monkey_list = []
    for m in input_monkey.keys():
        name = m
        items = input_monkey[m][1].split(': ')[1]
        clean_items = ([int(x.strip()) for x in items.split(',')])
        op_str = input_monkey[m][2].split(': ')[1]
        test_d = input_monkey[m][3].split('by ')[1]
        true_move = input_monkey[m][4].split('monkey ')[1]
        false_move = input_monkey[m][5].split('monkey ')[1]

        # build monkey
        monkey_list.append(monkey(name = name, 
                      items = clean_items, 
                      operation_str = op_str, 
                      test_divisor = int(test_d), 
                      true =  int(true_move), 
                      false = int(false_move), 
                      inspection_count=0,
                      anxiety_int = divider)
                      )
    
    for r in range(1,21):
        for m in monkey_list:
            m.parse_operation()
            m.review_items(monkey_list)

    if (monkey_list[0].inspection_count == 99) and (monkey_list[1].inspection_count == 97) and (monkey_list[2].inspection_count == 8) and (monkey_list[3].inspection_count == 103):
        test_d_list.append(divider)
        
        
    if divider % 10_000 == 0:
        print(f"Finished step {divider}: {len(test_d_list)} possible dividers")
        
print(f"Move to next stage w/ {len(test_d_list)} possible dividers")

Finished step 10000: 15 possible dividers
Finished step 20000: 52 possible dividers
Finished step 30000: 85 possible dividers
Finished step 40000: 113 possible dividers
Finished step 50000: 143 possible dividers
Finished step 60000: 172 possible dividers
Finished step 70000: 201 possible dividers
Finished step 80000: 224 possible dividers
Finished step 90000: 261 possible dividers
Move to next stage w/ 287 possible dividers


In [4]:
# now we have limited, go to next option
new_d_list = []
for divider in test_d_list:
    monkey_list = []
    for m in input_monkey.keys():
        name = m
        items = input_monkey[m][1].split(': ')[1]
        clean_items = ([int(x.strip()) for x in items.split(',')])
        op_str = input_monkey[m][2].split(': ')[1]
        test_d = input_monkey[m][3].split('by ')[1]
        true_move = input_monkey[m][4].split('monkey ')[1]
        false_move = input_monkey[m][5].split('monkey ')[1]

        # build monkey
        new_m = monkey(name = name, 
                                  items = clean_items, 
                                  operation_str = op_str, 
                                  test_divisor = int(test_d), 
                                  true =  int(true_move), 
                                  false = int(false_move), 
                                  inspection_count=0,
                                  anxiety_int = divider
                      )

        monkey_list.append(new_m)

    for r in range(1,1001):
        for m in monkey_list:
            m.parse_operation()
            m.review_items(monkey_list)

    if (monkey_list[0].inspection_count == 5204) and (monkey_list[1].inspection_count == 4792) and (monkey_list[2].inspection_count == 199) and (monkey_list[3].inspection_count == 5192):
        new_d_list.append(divider)
 
print(f"Move to next stage w/ {len(new_d_list)} possible dividers")
print(new_d_list)

Move to next stage w/ 1 possible dividers
[96577]


### With Actual Data:

In [5]:
with open('data/day11.txt') as fh:
    data = [line.strip() for line in fh.readlines()]
    
input_monkey = defaultdict(list)
i = 0
for line in data:
    if line == '':
        i += 1
    else:
        input_monkey[i].append(line)
        
monkey_list = []
for m in input_monkey.keys():
    name = m
    items = input_monkey[m][1].split(': ')[1]
    clean_items = ([int(x.strip()) for x in items.split(',')])
    op_str = input_monkey[m][2].split(': ')[1]
    test_d = input_monkey[m][3].split('by ')[1]
    true_move = input_monkey[m][4].split('monkey ')[1]
    false_move = input_monkey[m][5].split('monkey ')[1]

    # build monkey
    new_m = monkey(name = name, 
                              items = clean_items, 
                              operation_str = op_str, 
                              test_divisor = int(test_d), 
                              true =  int(true_move), 
                              false = int(false_move), 
                              inspection_count=0,
                              anxiety_int = 9699690
                  )

    monkey_list.append(new_m)

for r in range(1,10_001):
    for m in monkey_list:
        m.parse_operation()
        m.review_items(monkey_list)


final_list = []
for monkey in monkey_list:
    print(f"Monkey {monkey.name} inspected items {monkey.inspection_count}")
    final_list.append(monkey.inspection_count)
    
vals = sorted(final_list)
vals[-1] * vals[-2]

Monkey 0 inspected items 115025
Monkey 1 inspected items 93029
Monkey 2 inspected items 114545
Monkey 3 inspected items 114164
Monkey 4 inspected items 8590
Monkey 5 inspected items 30465
Monkey 6 inspected items 122693
Monkey 7 inspected items 122667


15050382231